<a href="https://colab.research.google.com/github/anahigomez/CadenaSuministro-AI/blob/master/c_Simulacion_discreta(simpy)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import math
!pip install simpy 
import simpy

In [0]:
# Atención en el Banco 
SEMILLA = 20
NUM_CAJEROS = 3
TIEMPO_ATENCION_MIN = 15
TIEMPO_ATENCION_MAX = 35
T_LLEGADAS = 10
TIEMPO_SIMULACION = 90
TOT_CLIENTES = 5
 
te  = 0.0 # tiempo de espera total
dt  = 0.0 # duracion de servicio total
fin = 0.0 # minuto en el que finaliza

In [3]:
# Funcion de atencion al clients
def Atencion(cliente):
    global dt  #Accedemos a la variable dt declarada anteriormente
    # numero aleatorio y lo guarda en R
    R = random.random()  
    tiempo = TIEMPO_ATENCION_MAX - TIEMPO_ATENCION_MIN  
    # Distribucion uniforme
    tiempo_aten = TIEMPO_ATENCION_MIN + (tiempo*R) 
    # deja correr el tiempo n minutos
    yield env.timeout(tiempo_aten) 
    print("3er paso: Atencion listo a %s en %.2f minutos" % (cliente,tiempo_aten))
    # Acumula los tiempos de uso de la i
    dt = dt + tiempo_aten 
#
def cliente (env, nombre, personal ):
    global te
    global fin
    # Guarda el minuto de llegada del cliente
    llega = env.now 
    print ("1ro paso: %s llego al Banco en el minuto %.2f" % (nombre, llega))
    # Espera su turno
    with personal.request() as request: 
        # Obtiene turno
        yield request 
        # Guarda el minuto cuando comienza a ser atendido
        pasa = env.now 
        # Calcula el tiempo que espero
        espera = pasa - llega 
        # Acumula los tiempos de espera
        te = te + espera 
        print ("2do paso: %s pasa con el cajero en el minuto %.2f habiendo esperado %.2f" % (nombre, pasa, espera))
        # Invoca al proceso Atencion
        yield env.process(Atencion(nombre)) 
        #Guarda el minuto en que termina el proceso Atencion 
        deja = env.now 
        print ("4to paso: %s deja el Banco en el minuto %.2f" % (nombre, deja)) 
        # Conserva globalmente el ultimo minuto de la simulacion
        fin = deja 

def principal (env, personal):
    llegada = 0
    i = 0
    # Para 10 clientes
    for i in range(TOT_CLIENTES): 
        R = random.random()
        # Distribucion exponencial
        llegada = -T_LLEGADAS * math.log(R) 
        # Deja transcurrir un tiempo entre uno y otro
        yield env.timeout(llegada)  
        i += 1
        env.process(cliente(env, 'Cliente %d' % i, personal)) 

print ("============= SIMULACION COLA EN EL BANCO=============")
 # Cualquier valor
random.seed (SEMILLA) 
# Crea el objeto entorno de simulacion
env = simpy.Environment() 
#Crea los recursos 
personal = simpy.Resource(env, NUM_CAJEROS) 
#Invoca el proceso princial
env.process(principal(env, personal))
#Inicia la simulacion   
env.run()       

============= SIMULACION COLA EN EL BANCO=============
1ro paso: Cliente 1 llego al Banco en el minuto 0.99
2do paso: Cliente 1 pasa con el cajero en el minuto 0.99 habiendo esperado 0.00
1ro paso: Cliente 2 llego al Banco en el minuto 4.76
2do paso: Cliente 2 pasa con el cajero en el minuto 4.76 habiendo esperado 0.00
1ro paso: Cliente 3 llego al Banco en el minuto 5.76
2do paso: Cliente 3 pasa con el cajero en el minuto 5.76 habiendo esperado 0.00
1ro paso: Cliente 4 llego al Banco en el minuto 10.29
1ro paso: Cliente 5 llego al Banco en el minuto 11.66
3er paso: Atencion listo a Cliente 2 en 20.20 minutos
4to paso: Cliente 2 deja el Banco en el minuto 24.95
2do paso: Cliente 4 pasa con el cajero en el minuto 24.95 habiendo esperado 14.66
3er paso: Atencion listo a Cliente 1 en 30.33 minutos
4to paso: Cliente 1 deja el Banco en el minuto 31.32
2do paso: Cliente 5 pasa con el cajero en el minuto 31.32 habiendo esperado 19.66
3er paso: Atencion listo a Cliente 3 en 33.10 minutos
4to pa

In [4]:
print ("\n---------------------------------------------------------------------")
print ("\nINDICADORES: ")
lpc = te / fin
print ("\nLongitud promedio de la cola: %.2f" % lpc)
tep = te / TOT_CLIENTES
print ("Tiempo de espera promedio = %.2f" % tep)
upi = (dt / fin) / NUM_CAJEROS
print ("Uso promedio de la instalacion = %.2f" % upi)
print ("\n---------------------------------------------------------------------")


---------------------------------------------------------------------

INDICADORES: 

Longitud promedio de la cola: 0.67
Tiempo de espera promedio = 6.87
Uso promedio de la instalacion = 0.83

---------------------------------------------------------------------
